<a href="https://colab.research.google.com/github/veren4/SMILES_featurization/blob/master/LSTM_KDNuggets_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook ist based on [this tutorial](https://www.kdnuggets.com/2020/07/pytorch-lstm-text-generation-tutorial.html). The code is from their Github repo.

Open problems with this model:
* Wenn ich in dem Satz, den ich hinten reinfüttere zum Predicten, ein Zeichen habe, das im Trainings-Datensatz nicht vorkam, kriege ich einen Fehler. => Generell muss ich unknown tokens einführen.
* Ich schaue das Vokabular des ganzen Datensatzes an. Wenn ich den aber am Anfang nicht einlese, geht das nicht => Vorher bestimmen und hier nur einlesen!
* Datensatz einlesen, ohne komplett in den Cache zu laden ([Massive Dataset class](https://github.com/pytorch/text/issues/130))
* Adapt lstm size and embedding dim?

#Dataset

In [ ]:
import torch
import pandas as pd
from collections import Counter

class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        args,
    ):
        self.args = args
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        #train_df = pd.read_csv('data/reddit-cleanjokes.csv')
        train_df = pd.read_csv('data/worded_smiles.csv')
        
        #infile = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles.csv'
        #with open(infile, "r") as file1:
        #    train_df = pd.read_csv(file1)
        #file1.close()
        
        text = train_df['Joke'].str.cat(sep=' ')
        return text.split(' ')

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.args.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.args.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.args.sequence_length+1]),
        )

#Model

#Train

In [7]:
!cp "drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/dataset.py" .
import dataset

In [2]:
!cp "drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/model.py" .
import model

In [3]:
!cp "drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/train_without_argparse.py" .
import train_without_argparse

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import torch
import platform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('Python: ', platform.python_version())
print('PyTorch: ', torch.__version__)
if(device.type == 'cuda'):
  print('Using GPU (cuda)')
else:
  print('Using CPU!')

Python:  3.6.9
PyTorch:  1.7.0+cu101
Using GPU (cuda)


In [8]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--max-epochs', type=int, default=10)
parser.add_argument('--batch-size', type=int, default=256)
parser.add_argument('--sequence-length', type=int, default=4)
args = parser.parse_args("")

dataset = dataset.Dataset(args)
model = model.Model(dataset)

FileNotFoundError: ignored

Change the dataset in dataset.py.\
Set max epochs, batch size, seqence length in train.py.

In [ ]:
from train_without_argparse import train
from train_without_argparse import predict

In [ ]:
state_h, state_c = train(dataset, model, args)
print(predict(dataset, model, text='C C ( O 2 ) C'))

In [ ]:
print(predict(dataset, model, text='C C ( O 2 ) C'))

In [ ]:
state_h

In [ ]:
state_h.d

In [ ]:
state_c